### Import Libraries

In [21]:
import os
import re
import string
import json
import emoji
import numpy as np
import pandas as pd
from sklearn import metrics
from sklearn.model_selection import train_test_split
from bs4 import BeautifulSoup
import transformers
import torch
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from transformers import AutoTokenizer, AutoModel, AdamW, BertTokenizer, BertModel, BertConfig
import warnings
warnings.filterwarnings('ignore')

pd.set_option("display.max_columns", None)

device = 'cuda' if torch.cuda.is_available() else 'cpu'

### Import the Dataset

In [22]:
data = pd.read_csv('go_emotions_dataset.csv')

with open('emotion_mapping.json') as file:
    emotion_mapping = json.load(file)

emotion_file = open('emotions.txt', "r")
emotion_list = emotion_file.read()
emotion_list = emotion_list.split("\n")

### Data Preprocessing

In [23]:
# get rid of unclear examples & associated column
unclear = data['example_very_unclear'] == True
data = data[~unclear]
data = data.drop(columns = ['example_very_unclear'])

In [24]:
# combine the rows with the same id
data = data.groupby('id').agg(lambda x: x.tolist())

# fix the text column
for row, col in data.iterrows():
    data.loc[row, 'text'] = data.loc[row, 'text'][0]

KeyboardInterrupt: 

In [ ]:
# boolean for emotion of each text
for row, _ in data.iterrows():
    for col in data.columns[1:]:
        if 1 in data.loc[row, col]:
            data.loc[row, col] = 1
        else:
            data.loc[row, col] = 0

In [ ]:
# dataframe for the 27 emotions + neutral
data28 = data.reset_index()
data28 = data28.drop(columns = ['id'])

In [ ]:
# write the name of the emotion(s) in the emotion column
data['Emotions'] = ''
for row, col in data.iterrows():
    for i in emotion_list:
        if col[i] == 1:
            if data.loc[row, 'Emotions'] != '':
                data.loc[row, 'Emotions'] += ' ' + i
            else:
                data.loc[row, 'Emotions'] = i

In [ ]:
# make a list for each row's emotion(s)
data['Emotions'] = data['Emotions'].apply(lambda x: x.split(' '))

In [ ]:
# map emotion to emotion category
def EmotionMap(emotion_list):
    map_list = []
    for i in emotion_list:
        if i in emotion_mapping['anger']:
            map_list.append('anger')
        if i in emotion_mapping['disgust']:
            map_list.append('disgust')
        if i in emotion_mapping['fear']:
            map_list.append('fear')
        if i in emotion_mapping['joy']:
            map_list.append('joy')
        if i in emotion_mapping['sadness']:
            map_list.append('sadness')
        if i in emotion_mapping['surprise']:
            map_list.append('surprise')
        if i == 'neutral':
            map_list.append('neutral')
    return map_list

In [ ]:
# map the emotion to the emotion category of emotions
data['Mapped Emotions'] = data['Emotions'].apply(EmotionMap)

In [ ]:
# list of emotion categories
emotion_cats = ['anger', 'disgust', 'fear', 'joy', 'sadness', 'surprise', 'neutral']
needed_cols = ['id', 'text', 'Emotions', 'Mapped Emotions']

# drop the unwanted columns
not_needed = []
for i in data.columns:
    if i not in emotion_cats and i not in needed_cols:
        not_needed.append(i)

# new dataframe to work with
df = data.drop(not_needed, axis=1)

In [ ]:
for i in emotion_cats:
    df[i] = df['Mapped Emotions'].apply(lambda x: 1 if i in x else 0)

### Clean the Text

In [ ]:
contraction_mapping = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not", 
                       "didn't": "did not",  "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not", 
                       "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", 
                       "how's": "how is",  "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am",
                       "I've": "I have", "i'd": "i would", "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", 
                       "i've": "i have", "isn't": "is not", "it'd": "it would", "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have",
                       "it's": "it is", "let's": "let us", "ma'am": "madam", "mayn't": "may not", "might've": "might have","mightn't": "might not",
                       "mightn't've": "might not have", "must've": "must have", "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", 
                       "needn't've": "need not have","o'clock": "of the clock", "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not",
                       "sha'n't": "shall not", "shan't've": "shall not have", "she'd": "she would", "she'd've": "she would have", "she'll": "she will", 
                       "she'll've": "she will have", "she's": "she is", "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have",
                       "so've": "so have","so's": "so as", "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is",
                       "there'd": "there would", "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would",
                       "they'd've": "they would have", "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have",
                       "to've": "to have", "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have",
                       "we're": "we are", "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", 
                       "what're": "what are",  "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did",
                       "where's": "where is", "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", 
                       "who've": "who have", "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have", 
                       "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all", "y'all'd": "you all would",
                       "y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have","you'd": "you would", "you'd've": "you would have",
                       "you'll": "you will", "you'll've": "you will have", "you're": "you are", "you've": "you have", 'u.s':'america', 'e.g':'for example'}

punct = [',', '.', '"', ':', ')', '(', '-', '!', '?', '|', ';', "'", '$', '&', '/', '[', ']', '>', '%', '=', '#', '*', '+', '\\', '•',  '~', '@', '£', 
 '·', '_', '{', '}', '©', '^', '®', '`',  '<', '→', '°', '€', '™', '›',  '♥', '←', '×', '§', '″', '′', 'Â', '█', '½', 'à', '…', 
 '“', '★', '”', '–', '●', 'â', '►', '−', '¢', '²', '¬', '░', '¶', '↑', '±', '¿', '▾', '═', '¦', '║', '―', '¥', '▓', '—', '‹', '─', 
 '▒', '：', '¼', '⊕', '▼', '▪', '†', '■', '’', '▀', '¨', '▄', '♫', '☆', 'é', '¯', '♦', '¤', '▲', 'è', '¸', '¾', 'Ã', '⋅', '‘', '∞', 
 '∙', '）', '↓', '、', '│', '（', '»', '，', '♪', '╩', '╚', '³', '・', '╦', '╣', '╔', '╗', '▬', '❤', 'ï', 'Ø', '¹', '≤', '‡', '√', ]

punct_mapping = {"‘": "'", "₹": "e", "´": "'", "°": "", "€": "e", "™": "tm", "√": " sqrt ", "×": "x", "²": "2", "—": "-", "–": "-", "’": "'", "_": "-",
                 "`": "'", '“': '"', '”': '"', '“': '"', "£": "e", '∞': 'infinity', 'θ': 'theta', '÷': '/', 'α': 'alpha', '•': '.', 'à': 'a', '−': '-', 
                 'β': 'beta', '∅': '', '³': '3', 'π': 'pi', '!':' '}

mispell_dict = {'colour': 'color', 'centre': 'center', 'favourite': 'favorite', 'travelling': 'traveling', 'counselling': 'counseling', 'theatre': 'theater',
                'cancelled': 'canceled', 'labour': 'labor', 'organisation': 'organization', 'wwii': 'world war 2', 'citicise': 'criticize', 'youtu ': 'youtube ',
                'Qoura': 'Quora', 'sallary': 'salary', 'Whta': 'What', 'narcisist': 'narcissist', 'howdo': 'how do', 'whatare': 'what are', 'howcan': 'how can',
                'howmuch': 'how much', 'howmany': 'how many', 'whydo': 'why do', 'doI': 'do I', 'theBest': 'the best', 'howdoes': 'how does', 
                'mastrubation': 'masturbation', 'mastrubate': 'masturbate', "mastrubating": 'masturbating', 'pennis': 'penis', 'Etherium': 'Ethereum', 
                'narcissit': 'narcissist', 'bigdata': 'big data', '2k17': '2017', '2k18': '2018', 'qouta': 'quota', 'exboyfriend': 'ex boyfriend', 
                'airhostess': 'air hostess', "whst": 'what', 'watsapp': 'whatsapp', 'demonitisation': 'demonetization', 'demonitization': 'demonetization',
                'demonetisation': 'demonetization'}

In [ ]:
def clean_text(text):
    # clean emojis
    text = emoji.demojize(text)
    text = re.sub(r'\:(.*?)\:', '' ,text)

    # make lowercase
    text = str(text).lower()
    text = re.sub('\[.*?\]', '', text)

    # remove HTML text
    text = BeautifulSoup(text, 'lxml').get_text()
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    text = re.sub(r"[^a-zA-Z?.!.¿']+", " ", text)
    return text

def clean_contractions(text, mapping):
    #specials = ["’", "‘", "´", "`"]
    #for s in specials:
    #    text = text.replace(s, "'")
    for word in mapping.keys():
        text = text.replace(""+word+"", ""+mapping[word]+"")

    # remove punctuation
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)

    # add space between word and punctuation
    text = re.sub(r"([?.!,¿])", r" \1 ", text)
    text = re.sub(r'[" "]+', " ", text)
    return text

def clean_special_characters(text, punct, mapping):
    for p in mapping:
        text = text.replace(p, mapping[p])

    for p in punct:
        text = text.replace(p, f' {p} ')
    
    specials = {'\u200b': ' ', '…': ' ... ', '\ufeff': '', 'करना': '', 'है': ''}
    for s in specials:
        text = text.replace(s, specials[s])
    return text

def correct_spelling(x, dic):
    for word in dic.keys():
        x = x.replace(word, dic[word])
    return x

def remove_spaces(text):
    text = text.strip()
    text = text.split()
    return " ".join(text)

def text_preprocessing(text):
    text = clean_text(text)
    text = clean_contractions(text, contraction_mapping)
    text = clean_special_characters(text, punct, punct_mapping)
    text = correct_spelling(text, mispell_dict)
    text = remove_spaces(text)
    return text

In [ ]:
# clean text of dataset with 6 emotions + neutral
df['text'] = df['text'].apply(text_preprocessing)

# clean text of dataset with 27 emotions + neutral
data28['text'] = data28['text'].apply(text_preprocessing)

### Split into Train and Test Datasets

In [ ]:
# split the dataframe with 6 emotions + neutral
X = df.iloc[:, 0] # tokenized text
y = df.iloc[:, 1:8] # labels

# split data into training and testing datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3)

# combine the text with the labels
df_train = pd.concat((X_train, y_train), axis = 1)
df_test = pd.concat((X_test, y_test), axis = 1)

# reset the index of the dataframes
df_train = df_train.reset_index(drop = True)
df_test = df_test.reset_index(drop = True)

# save the dataframes to csv files
df_train.to_csv('6emotions/train6.csv', index = False)
df_test.to_csv('6emotions/test6.csv', index = False)

In [ ]:
# split the dataframe with 27 emotions + neutral
X = data28.iloc[:, 0] # tokenized text
y = data28.iloc[:, 1:] # labels

# split data into training and testing datasets
X_train27, X_test27, y_train27, y_test27 = train_test_split(X, y, test_size = 30)

# combine the text with the labels
df_train27 = pd.concat((X_train27, y_train27), axis = 1)
df_test27 = pd.concat((X_test27, y_test27), axis = 1)

# reset the index of the dataframes
df_train27 = df_train27.reset_index(drop = True)
df_test27 = df_test27.reset_index(drop = True)

# save the dataframes to csv files
df_train27.to_csv('27emotions/train27.csv', index = False)
df_test27.to_csv('27emotions/test27.csv', index = False)